# Introduction to Relational Databases and SQL

This notebook serves as an introduction to relational databases and SQLite for first year students in the bachelor Computational Social Sciences at the University of Amsterdam. It assumes previous knowledge in Python, but no knowledge in SQL is required. The notebook has been created by Néstor Narbona Chulvi, and it uses material by <a heref = 'https://towardsdatascience.com/designing-a-relational-database-and-creating-an-entity-relationship-diagram-89c1c19320b2'> Craig Dickson </a> and a database from <a href = 'https://www.sqlitetutorial.net/about-us/'> SQLite tutorial </a>. 

The first 5 sections are an introduction to different conceptual topics as well as a programming tutorials. They all include self-contained theory as well as short exercises to put theory in to practice. The last section is a bigger exercise where students will have make use of many different concepts and methods introduced throughout the notebook. 

**Table of contents**
1. Introduction to relational databases 
2. What is SQL and how to set it up
3. Working with SQLite: creating a relational database
4. Working with SQLite: data analysis on a relational database
5. Integrating SQLite and Python
6. Exercise: create your own relational database


### 1. Introduction to Relational Databases

**What is a relational database?**

A relational database (RD) is a type of database that stores and provides access to data points that are related to one another (we will shortly see what this 'related to each other' really means). In a relational database, information is stored in tables (notice the plural, as a database may contain one or more tables, usually plenty), where each column is an attribute and each row is a record (i.e. a particular observation). See for example Table 1, where the different pens a shop sells are the entries, and each of them has different attributes.

TABLE 1: 
| **product_id** | brand | type |product_name | price |
| --- | --- | --- | --- | --- |
| **1** | Bic | Pen | Bic Crystal | 1.5 |
| **2** | Bic | Pen | Bic Shiny | 2 |
| **3** | Montblanc | Fountain Pen |Gentleman | 75 |
| **4** | Pilot | Pen | Gel Pen | 3 |
| **5** | Thomas | Pen| Gel Pen |2.5|
 

Tables in a RD have some important components. Firstly, we find the *primary key*, which is an attribute that uniquely identifies a record (i.e. a row). One may also create a composite *primary key*, in which case it will be a combination of attributes which uniquely identify a record. What is crucial is that every table in a RD must have a *primary key*, which will have unique values and will not be NULL (i.e. NULL is equivalent to NA in pandas). In our example from Table 1, we have an attribute **product_id** which acts as primary key, as it uniquely identifies each record. 

What makes RD 'relational' is the way different tables in a database are connected. Lets for instance introduce Table 2 into our database, where information about the pen manufacturers is stored. We say that we have two tables, each for a particular *entity* (i.e. the pens sold, and the pen manufacturers). In Table 2 we again observe a primary key, in this case the brand name. We are expressing the relationship between the two tables by linking each pen with its manufacturer, by including the brand attribute in Table 1. When we include the primary key from a another table into our table, we call it a *foreign key*. 

Another important point to notice is that one brand may produce more than one pen, while each pen may only be produced by one brand. This is an example of a 1-to-N relationship, but we may also observe 1-to-1 or N-to-M relationships. This are referred to as the *cardinality of the relationship*. We will later see how to handle N-to-M cardinalities. For the time being, notice that when we observe a 1-to-N cardinality we store the foreign key in the N side of the relationship. In our particular case, we store the brand in the pen table, and not the other way around. 

TABLE 2: 
| **brand** | country | contact_person |
| --- | --- | --- |
| **Bic** | France | Monsieur Poincaré | 
| **Montblanc** | Switzerland | Franca Roulleau |
| **Pilot** | Japan | Akira Kurosawa | 
| **Thomas** | Spain | Paco Leon |

**Questions**
- Why is it imperative for a table to have a primary key? Think what would happen if there was not a column with such characteristics as the primary key. 
- When we observe a 1-to-N cardinality, we store the foreign key in the entity that may only have one relation to the other entity. Can tell why we do this? Think of what we would happen if we were to store the produced pens in the brand table, do you see any redundancies? 

**Answer**
- If a table did not have a primary key (i.e. there would be no column with unique non-null values), it is not guaranteed that you can read (i.e. select) all records. If two pens have the same id and the rest of variables are equal you could not differentiate them. And even if the rest of the values were not equal, one would have to look at the different values and see how to index this particular record. All this trouble is avoided by having a primary key. 

- We would have to add as many entries are there linked records on the other table, thus adding new data points that are unnecessary. In the particular case of the brand table, we would need to add a row for Bic and have something like this: 

| **brand** | country | contact_person | product | 
| --- | --- | --- | --- |
| **Bic** | France | Monsieur Poincaré | Bic crystal |
| **Bic** | France | Monsieur Poincaré | Bic Shyni |
...


**Entity relationship diagrams**

In reality we design RDs with many more entities and more complicated relationships between them. It is then very useful to make use of some sort of diagram which clearly specifies which *entities* constitute our database and how do they relate to each other. This tool is referred to as an Entity Relationship Diagram (ERD). The collection of entities and relationships that will form a RD is referred to as the *requirements* of the database. The job when designing a RD is to satisfy these requirements. Lets see complicate our stationary example and use it to introduce ERD. 

> **Requirements for the stationary database:** We have different pens which are manufactured by one brand and sold in one or more shops. A brand manufactures one or more pens, and owns at least one shop.

We are required to create a database to store this information. The first natural question is how many entities we have. In this case we have: pens, shops and manufacturers. For each of these entities we will want to store some attributes such as the ones we store in tables 1 and 2. And finally, we will have to relate them in the correct way. All of these can be synthesized in an ERD, such as the one in Figure 1. Lets break it down. 

Figure 1:

![ERD for the described requirements](diagrams/diagram.jpg)

Firstly, we see that each entity is represented by a box, where its attributes are shown. We mark in red the primary key and foreign keys are marked green. All of these we have already seen. But in this diagram we are also representing the relationship between the different entities. We do this by connecting them with lines with a 'meaningful' verb describing what the relationship consists of. On top of that we see the cardinality represented next to this verbs. For instance, the relationship we already discussed between pens and brands, a 1-to-N cardinality, is represented by an N on the pens side and a 1 on the brands side. Because a shop may sell many pens and a pen may be sold in many shops, we have an N-to-M relationship. You will notice that there are no foreign keys representing this relation, we will see later why this is and how one represents this sort of relationships in a RD. Finally, some lines are double and some are not. This represents the *participation level*. If all the records of an entity participate in a relationship, the line is double. In the 'brands own shops' relationship this is the case for brands, as they all own at least one shp. It is not the case for shops, though, as the may not be owned by a pen brand. 

**Questions**
- Imagine we add the following requirement to our database: "*We also keep track of employees. They work for one of the brands and they specialize in one pen*". How would you incorporate this into the ERD shown in Figure 1? 

**Answers**
Ignoring what particular attributes employee may have: 

![ERD for the described requirements](diagrams/diagram2.jpg)

### 2. What is SQL and how to set it up

We have introduced relational databases and introduced its conceptual backbone. But how do we actually implement these ideas computationally? The answer is Structure Query Language (SQL). SQL is a programming language used to create and manage RDs, and it will allow us to incorporate all the ideas we have discussed in the previous section. One particularity of SQL is that there different *dialects*  (e.g. Oracle, Microsoft SQL Server, MySQL). Although they are all built upon SQL and incorporate the same concepts, the syntax is slightly different. It is not usually possible to use SQL code written for PostgreSQL in Microsoft SQL Server, for example, without making some modifications. For this notebook we will use SQLite, and extremely light-weight and serverless (i.e. runs on your machine and directly acts on the database) version with almost non needed set up procedure. You can find the official documentation for SQLite <a href = 'https://www.sqlite.org/docs.html'> here </a>.

**Installing SQLite**

In most modern computers (Windows 10 or higher) SQLite comes pre-installed. To check if this is the case in your computer, open the terminal and run the command `sqlite3`. If you see a response showing the version number among other information, the SQLite is already installed. If the terminal does not recognize the command, then you still need to install it. For this we recommend you follow the steps in <a href = https://sqldocs.org/sqlite/sqlite-installation/#pre-installation-checks>this tutorial</a>.

**Setting up a workspace for SQLite**

When working with RDs using SQL languages you will mostly encounter two types of file extensions `.db` and `.sql`. The former will be the file where the database is stored. The latter will be the file where SQL code is written. You can think of this as having a `.csv` file with your data and and `.py` file with your Python code that reads and changes the data file. 

Like we saw with Python, one could write SQLite code directly on the terminal. But because this is quite inconvenient we use an IDE to write and run our code. There are many options for this, but we will try to keep it simple and use VisualStudio together with <a href = 'https://marketplace.visualstudio.com/items?itemName=alexcvzz.vscode-sqlite'> this </a> SQLite extension. 

Once you have SQLite in your computer and you have downloaded the SQLite extension for Visual Studio, move to the next step. 


### 3. Working with SQLite: Creating a relational database

We are first going to use SQLite to create a relational database. To make things more interesting, we will use a new set of requirements for our database, involving the organization of a Language School:

*The International Language School provides language training services for corporate clients, offering classes either at their premises or at the School itself. The School employs multiple teachers, each capable of teaching various courses. Clients, in turn, offer courses to their employees, who can enroll in multiple courses. Each course is associated with one client and one teacher, and participants are employees of the client companies, with the possibility of being enrolled in more than one course.*

I encourage you to try and create an ERD for yourself based on these requirements. The one we will be using to guide the creating of our database can be seen in Figure 3: 

Figure 3: 

![ERD for the described requirements](diagrams/diagram_schools.jpg)

Our first step now is to create the database file that will contain this RD. For that, create a new directory and open it in VisualStudio (File + Open Folder). Then create a new file in this directory called `school_database.db`. Now, open the file, right click on it and press 'new query'. An SQL file will open on a new tab, and here is where we will be writing our code. Save this file in the same directory and as `school_create.sql`. 



**Creating tables** 

> You are about to see a lot of SQL code. Keep in mind two things, although we capitalize all SQL keywords this is not necessary for the code to run. Nevertheless, it is standard practice for readability purposes. Similarly, line changes and indents are irrelevant, but it is standard to include them for readability.

The first thing we will do is create the four tables for the four entities we have in our database. For this is crucial to get familiar with the different data types that SQL supports. There are many but here we will be using $^1$: 
- INT : an integer number (e.g. 0, 1, 2, 3...)
- NULL : a cell with missing data.
- TEXT : a string. 


With this in mind, look at the following code used to create the `client` table: 

```
CREATE TABLE client (
  client_id INT PRIMARY KEY,
  client_name TEXT NOT NULL,
  address TEXT NOT NULL,
  industry TEXT
);
```

We first use the key words `CREATE TABLE` to tell SQL that we want to create a new table. We then, give the name of the table `client` and open brackets to define the columns. For each column we specify the name (e.g. `client_id`) and the data type (e.g `INT`). We use a comma after defining each column and close the brackets when all the columns have been defined. Finally, we finish all SQL code with a `;`. 

There are some other key words following the data type in each column. This are referred to as *constraints*. With `NOT NULL` we specify that `client_name` and `address` cannot be left empty when imputing a record. We also specify which of our attributes is going to act as `PRIMARY KEY`. 

Now that we understand the code, lets run it and actually create this table in our `school_database.db`. To do so, copy the code to `school_create.sql`, double click on the file and press `Run query` (you may also use the shortcut Ctrl + Shift + Q). A drop down menu will show, asking you to select the `.db` file you want to run this code on, select `school_database.db`. From now onwards, the code you run on this file will take effect on this database. 

Congratulations! You have created your first SQL table. To check this you can run the code `.tables` to get a list of the tables in the dataset. To run new code using this file you have two options: comment out the previous piece of code and use `Run query` again (you will get an error if you run code that creates a table which already exists), or select the code you want to run, right click and press `Run Selected Query`. 

In case you wanted to delete the table (e.g. had you designed it wrong in some way), you can easily do so with the following code: 

```
DROP TABLE teacher;
```
Keep in mind that this permanently deletes the table **together with all its records**. So be very careful using this code in the future. 

$^1$: In fact SQLite does not require you to define the data type a column will contain. Nevertheless, because most other SQL dialects do, it is common practice to specify the data type to maximize compatibility. 

Now that we have defined the `client` table, we can define the `participant` table which has a foreign key related to it. Note that in `SQLite` one cannot define a table with a foreign key before defining the table the key refers to. The code for this new table will be mostly equivalent to the previous one, lets see it: 

```
CREATE TABLE participant (
  participant_id INT PRIMARY KEY,
  first_name TEXT NOT NULL,
  last_name TEXT NOT NULL,
  phone_no TEXT,
  client TEXT,
  FOREIGN KEY(client) REFERENCES client(client_id)
);
```

At the end of the code we are adding a new line. With `FOREIGN KEY` followed by the attribute name, we specify which attribute is going to function as a foreign key in this table. With `REFERENCES client(client_id)` we specify that this attribute corresponds to the `client_id` attribute in the `client` table. 

**Question**

Now that you know how to create a table, try to create the remaining two tables in the ERD. You should define the primary and foreign keys. Pay attention to the other in which you create these tables!

**Answer**

```
CREATE TABLE teacher (
  teacher_id INT PRIMARY KEY,
  first_name TEXT NOT NULL,
  last_name TEXT NOT NULL,
  language_1 TEXT NOT NULL,
  language_2 TEXT,
  dob TEXT,
  tax_id INT UNIQUE,
  phone_no TEXT
  );

CREATE TABLE course (
  course_id INT PRIMARY KEY,
  course_name TEXT NOT NULL,
  language TEXT NOT NULL,
  level TEXT,
  course_length_weeks INT,
  start_date TEXT,
  in_school TEXT,
  teacher TEXT,
  client TEXT,
  FOREIGN KEY(teacher) REFERENCES teacher(teacher_id)
  FOREIGN KEY(client) REFERENCES client(client_id)
);
```


We are almost done with the design of our database. But just as we saw in the first section, we have a relationship with cardinality N-to-M between `participants` and `courses`. In other words, one participant may take multiple courses and a course may be taken by multiple participants. This sort of relationships cannot be handled simply using foreign keys, instead we create a new table dedicated to handling the relationship between courses and participants. This table 'connects' the primary keys of the entities involved in the relationship. 

Lets understand this table with an example. Imagine there are three participants: Josh, Marta and Tom, and two courses English and Spanish. Marta takes English, Tom takes Spanish and Josh takes both. In this case, the table relating these two entities would look like this: 

|participant|course|
|---|---|
|Josh|English|
|Josh|Spanish|
|Marta|English|
|Tom|Spanish|

Only in SQL we would use `course_id` and `participant_id`. 

The SQLite code to create such table is: 

```
CREATE TABLE takes_course (
  participant_id INT,
  course_id INT,
  PRIMARY KEY(participant_id, course_id),
  FOREIGN KEY(participant_id) REFERENCES participant(participant_id),
  FOREIGN KEY(course_id) REFERENCES course(course_id)
);
```
Most elements here should be familiar. We have decided to call the table `takes_course`, and we have defined `participant_id` and `course_id` as foreign keys relating to the corresponding tables. The only novelty is that we have a composite primary key, since neither the `participant_id` nor the `course_id` will be unique. We define this using the key words `PRIMARY KEY` followed by the attributes that will form the composite key between parenthesis. 

**Populating tables**

Now that we have designed the structure of our database (i.e. its entities and all relations between them), we can start imputing data. For example, if we want to add a teacher's record into the `teacher` table we can use this code: 

```
INSERT INTO teacher VALUES
(1,  'James', 'Smith', 'ENG', NULL, '1985-04-20', 12345, '+491774553676');
```

We use the key words `INSERT INTO` followed by the table we want to insert the data in and `VALUES`. This is then followed by and ordered enumeration of the attributes of this particular record between parenthesis. Note that text must go between quotation marks, just like in Python, and that date is also considered a string and thus should also go between quotation marks. It is also easy to insert multiple records at once: 

```
INSERT INTO teacher VALUES
(1,  'James', 'Smith', 'ENG', NULL, '1985-04-20', 12345, '+491774553676'),
(2, 'Stefanie',  'Martin',  'FRA', NULL,  '1970-02-17', 23456, '+491234567890'), 
(3, 'Steve', 'Wang',  'MAN', 'ENG', '1990-11-12', 34567, '+447840921333');
```

Finally, we can also input incomplete records: 

```
INSERT INTO teacher (teacher_id, first_name, last_name, language_1)
VALUES (2, 'Stefanie',  'Martin',  'FRA');
```
Be sure to include all attributes with a `NOT NULL` constraint, or the input of data will fail. All attributes that have not been inserted will take value `NULL`

You can see the records you have added to the table by displaying all its content with the query: 

```
SELECT * FROM teacher;
```

Finally, lets see how to delete and update records. To delete the first teacher we added to the `teacher` table we would use: 

```DELETE FROM teacher WHERE teacher_id = 1```

Here we use the keywords `DELETE FROM` followed by the table where the record is and then the keyword `WHERE` followed by a condition that would uniquely identify the record you want to delete (i.e. the primary key corresponds to that of the record we want to delete). If we wanted to correct the surname of the second teacher we inserted we would use: 

```
UPDATE teacher
SET last_name = 'Jones'
WHERE teacher_id = 1;
```

Here we use the keyword `UPDATE` followed by the table where the record to be updated is. We then use `SET` followed by the attribute we want to change and the new value we want to give it. Finally, we again specify a condition that uniquely identifies the record we want to change. 

**Question**

- Introduce the information of a client named Feather Flights Inc., with address 27 Ersatz Allee, 10317 Berlin which specializes in the aeronautical industry? 

- Could you insert information about a course without specifying the language it is taught in?

- Now delete all records you have introduced up until now. (This will help you avoid errors in the next part of this section)

**Answers**

```
INSERT INTO client VALUES
(1, 'Feather Flights Inc.', '27 Ersatz Allee, 10317 Berlin', 'Auronautical')
```

No, since the attribute `language` has a `NOT NULL` constraint. 

We are now going to completely populate the database. For this, we will provide you the code to introduce the records manually. Make sure that your tables are empty, or you may run into errors (e.g. introducing a new record with the same primary key as an existing one). You can copy paste the code and run it. 

```
INSERT INTO teacher VALUES
(1,  'James', 'Smith', 'ENG', NULL, '1985-04-20', 12345, '+491774553676'),
(2, 'Stefanie',  'Martin',  'FRA', NULL,  '1970-02-17', 23456, '+491234567890'), 
(3, 'Steve', 'Wang',  'MAN', 'ENG', '1990-11-12', 34567, '+447840921333'),
(4, 'Friederike',  'Müller-Rossi', 'DEU', 'ITA', '1987-07-07',  45678, '+492345678901'),
(5, 'Isobel', 'Ivanova', 'RUS', 'ENG', '1963-05-30',  56789, '+491772635467'),
(6, 'Niamh', 'Murphy', 'ENG', 'IRI', '1995-09-08',  67890, '+491231231232');
INSERT INTO client VALUES
(101, 'Big Business Federation', '123 Falschungstraße, 10999 Berlin', 'NGO'),
(102, 'eCommerce GmbH', '27 Ersatz Allee, 10317 Berlin', 'Retail'),
(103, 'AutoMaker AG',  '20 Künstlichstraße, 10023 Berlin', 'Auto'),
(104, 'Banko Bank',  '12 Betrugstraße, 12345 Berlin', 'Banking'),
(105, 'WeMoveIt GmbH', '138 Arglistweg, 10065 Berlin', 'Logistics');
INSERT INTO participant VALUES
(101, 'Marina', 'Berg','491635558182', 101),
(102, 'Andrea', 'Duerr', '49159555740', 101),
(103, 'Philipp', 'Probst',  '49155555692', 102),
(104, 'René',  'Brandt',  '4916355546',  102),
(105, 'Susanne', 'Shuster', '49155555779', 102),
(106, 'Christian', 'Schreiner', '49162555375', 101),
(107, 'Harry', 'Kim', '49177555633', 101),
(108, 'Jan', 'Nowak', '49151555824', 101),
(109, 'Pablo', 'Garcia',  '49162555176', 101),
(110, 'Melanie', 'Dreschler', '49151555527', 103),
(111, 'Dieter', 'Durr',  '49178555311', 103),
(112, 'Max', 'Mustermann', '49152555195', 104),
(113, 'Maxine', 'Mustermann', '49177555355', 104),
(114, 'Heiko', 'Fleischer', '49155555581', 105);
INSERT INTO course VALUES
(12, 'English for Logistics', 'ENG', 'A1', 10, '2020-02-01', TRUE,  1, 105),
(13, 'Beginner English', 'ENG', 'A2', 40, '2019-11-12',  FALSE, 6, 101),
(14, 'Intermediate English', 'ENG', 'B2', 40, '2019-11-12', FALSE, 6, 101),
(15, 'Advanced English', 'ENG', 'C1', 40, '2019-11-12', FALSE, 6, 101),
(16, 'Mandarin für Autoindustrie', 'MAN', 'B1', 15, '2020-01-15', TRUE, 3, 103),
(17, 'Français intermédiaire', 'FRA', 'B1',  18, '2020-04-03', FALSE, 2, 101),
(18, 'Deutsch für Anfänger', 'DEU', 'A2', 8, '2020-02-14', TRUE, 4, 102),
(19, 'Intermediate English', 'ENG', 'B2', 10, '2020-03-29', FALSE, 1, 104),
(20, 'Fortgeschrittenes Russisch', 'RUS', 'C1',  4, '2020-04-08',  FALSE, 5, 103);
INSERT INTO takes_course VALUES
(101, 15),
(101, 17),
(102, 17),
(103, 18),
(104, 18),
(105, 18),
(106, 13),
(107, 13),
(108, 13),
(109, 14),
(109, 15),
(110, 16),
(110, 20),
(111, 16),
(114, 12),
(112, 19),
(113, 19);

```

### 4. Working with SQLite: Data analysis on a relational database

**Basic queries**

In the previous section we saw how to create a database and 'populate' it with records. We will now see how to access the database so the data contained in it can be analyzed. For this, lets create a new SQLite file in the directory where `school_database.db` is and call it `school_analysis.sql`. We have already seen two queries that allow us to get information from the database, namely `.tables` to see the name of all the tables in the database and `SELECT * FROM table_name` to see all the content in a table. But lets dive deeper. 

We refer to pieces of SQL code that allow us to access certain data from the database as *queries*. The basic structure of a query is: 

```
SELECT attributes
FROM table
WHERE condition;
```

In the query `SELECT * FROM table_name` we specify that we want to see all attributes using `*`, and we do not specify any condition, thus all the data is shown. We could also ask for a particular group of attributes: 

```
SELECT client_name, industry
FROM client;
```

> Notice that when writing queries, just like when we were creating the database, if we `Run query` all the code in the file will be ran. You can either comment out or delete previous queries, or use `Run selected query` instead. 

**Conditions**

You will rarely want to get all the data from a column, in most cases you want to filter the data in some way. For this we use conditions after the keyword `WHERE`. Notice that this conditions are also used when creating, manipulating or deleting data (remember how we used conditions when updating and deleting records in the previous section). Thus, all you are about to learn about conditions also apply to these situations. 

A basic example using conditions would be finding all English courses: 

```
SELECT * 
FROM course
WHERE language = 'ENG'
ORDER BY level;
```

Here only records where the attribute `language` is equal to `ENG` will be displayed. Notice that we also ordered the result by `level`. By default SQLite orders in a descending fashion, you can change this using the keyword `ORDER BY attribute DESC`. You can also apply ordering by multiple attributes, where the hierarchy of the order will follow the order of the attributes. Try for instance:

```
SELECT * 
FROM course
WHERE language = 'ENG'
ORDER BY level, course_length_weeks;
```

>It is crucial that you respect the order of the keywords, first SELECT, then FROM, then the condition.


**Questions**
- Show the first name of the teachers who's second language is English sorted alphabetically.


**Answers** 
```
SELECT first_name
FROM teacher
WHERE language_2 = 'ENG'
ORDER BY first_name;
```

Lets now introduce some key logical operators that are used to define more sophisticated conditions. 

- `AND`: Allows you to specify multiple conditions and they all must be met.
- `OR`: Specify multiple conditions and any of them must be met.
- `NOT`: The condition that follows must not be met.
- `IN`: Allows to specify a list of possible value the attribute could have. For instance first_name in ('Jacob', 'Marta', 'Thomas').
- `!=` : Stands for *not equal to*
- Inequalities: you may use any of the inequalities you use in python: `>, <, >=, <=`
- `IS NULL`: The attribute must be `NULL`. You can also use `IS NOT NULL`. 
- `BETWEEN`: Specify that a date or a numerical attribute mus fall within an specified range. For instances `dob BETWEEN '1990-01-01' AND '1999-12-31`
- `LIKE`: This operator allows you to use wildcards: text patterns that multiple different patterns may satisfy. Wildcards are built using the operators `%` and `_`. The first stands for any number of characters and the second for one single character. For instance, if we want to match any name that starts with 'j', we can use the condition `first_name LIKE 'J%'`.

**Questions**
- Select English courses with a length of 10 weeks
- Select courses teaching English, French or Russian which are shorter than 40 weeks. Sort the resulting courses from easiest to hardest.
- Select all teachers born in the 90s. Sort them from oldest to youngest.
- Select the course name and level of all courses which include the characters 'interm' anywhere in their name. 

**Answers**

```
SELECT * 
FROM course
WHERE language = 'ENG' AND course_length_weeks = 10;

SELECT *
FROM course
WHERE language in ('ENG','FRA','RUS') AND course_length_weeks < 40
ORDER BY level;

SELECT *
FROM teacher
WHERE dob BETWEEN '1990-01-01' AND '1999-12-31'
ORDER BY dob;

SELECT course_name, level
FROM course
WHERE course_name LIKE '%interm%';
```



**Aggregating data**

Selecting data is very useful and we will continue using it following sections. Nonetheless, we sometimes want to summarize the data we select into a single statistics (e.g. the mean, the variance, the maximum value). SQLite incorporates some aggregating methods to achieve this. We can for instance obtain the average course length with the following query: 

```
SELECT AVG(course_length_weeks)
FROM course;
```

You can read more about what aggregating methods SQLite includes <a href = 'https://www.sqlite.org/lang_aggfunc.html'> here </a>. It is specially useful to combine these with the `GROUP BY` keyword, to obtain the desired statistic for each value of an attribute. For instance, we can get the average course length per client: 

```
SELECT client, AVG(course_length_weeks)
FROM course
GROUP BY client;
```

**Questions**
- Create a query that outputs a table showing how many courses are taught in each language. (HINT: Use the aggregating method COUNT())
- Create a query that shows how many courses each participant follows. Sort the result so the participants that follow most courses are shown first.

**Answers**
```
SELECT language, COUNT(language)
FROM course
GROUP BY language;

SELECT participant_id, COUNT(participant_id)
FROM takes_course
GROUP BY participant_id
ORDER BY COUNT(participant_id) DESC;
```

**Nested queries**

Sometimes we need to use the information we obtain from a query in another query. Imagine, for instance, that I want to look up all the courses that Niamh Murphy teaches. First, I need to know what is the teacher id of Mr. Murphy. Once I have this information, I can query the `course` table filtering courses with a teacher with this id. By using nested queries we can do this in one go: 

```
SELECT course_name
FROM course 
WHERE teacher = (
    SELECT teacher_id
    FROM teacher 
    WHERE first_name = 'Niamh' AND last_name = 'Murphy');
```

Notice that the nested query is written between parenthesis and there is no need to finish it with a `;`. 

**Questions**
- Use a nested query to select the teachers that are younger than the average (HINT: You will need an aggregate method for this).

**Answer**
```
SELECT *
FROM teacher
WHERE dob > 
    (SELECT AVG(dob)
    FROM teacher);
```

**Inner joins**

So far we have used our RD and SQLite to create tables and query them. Nevertheless, the real strength of these approaches becomes clear when we start joining tables that are related, to obtain the information we want. A *join* means combining two tables based on a column they have in common. For instance, we can relate the `teacher` table to the `course` table, and obtain more information on the courses each teacher tutors, by means of the columns `teacher_id` in `teacher` and `teacher` in `course`. This is the main role of foreign keys. 

The default type of join in SQLite is the *inner joins*, which matches rows that have the same value on the matching column. For instance, we can use an inner join to get a table of all participants and what industry they work in: 

```
SELECT participant.first_name, participant.last_name,client.industry
FROM participant
JOIN client ON participant.client = client.client_id;
```

Notice that now we are working with multiple tables, and thus the columns after `SELECT` must be specified using the *table.column* syntax. We specify `FROM` as usual, choosing one of the two tables we are going to join. Finally, we include `JOIN` followed by the other table and `ON` followed by the two columns that should be equal. 

>Which table you use in the `FROM` statement and which in the `JOIN` statement does not change the result. 

You can combine joins with all the concepts we have introduced before, such as conditions:

```
SELECT participant.first_name, participant.last_name
FROM participant
JOIN client ON participant.client = client.client_id
WHERE client.industry = 'NGO';
```

Finally, you may be wondering how we deal with N-to-M relationships. The answer is simple, we first join one of the tables to the *relationship table* (i.e. `takes_course` in our case), and then we join again to the other table. For instance, if we want to get the names of all participants who are taking a course of level B2, we would use a query like this: 

```
SELECT participant.first_name, participant.last_name
FROM participant
JOIN takes_course ON takes_course.participant_id = participant.participant_id
JOIN course ON course.course_id = takes_course.course_id
WHERE course.level = 'B2';
```

>If you struggle to see why the previous query makes sense, try joining `participant` and `takes_course` first, and look at the entire table. Then join also `course` and look at the entire table again. 

**Questions**
- Write a query that returns a table with all the courses that will not be taught in the school together with the address where they will be taught (i.e. the address of the client that bought the course).

- Write a query that returns a table with the courses that employees from NGOs study (HINT: You will observe repeated rows)

**Answers**
```
SELECT course.course_name, client.address
FROM client
JOIN course
ON course.client = client.client_id
WHERE course.in_school = FALSE;


SELECT course.course_name
FROM course
JOIN takes_course ON takes_course.course_id = course.course_id
JOIN participant ON participant.participant_id = takes_course.participant_id
WHERE participant.client = (
    SELECT client_id
    FROM client
    WHERE industry = 'NGO');
```

**Outer joins**

The type of join we have studied so far, the inner join, only displays rows that have a matching value in the column from both tables. If there a row has a certain value that has no equivalent in the other table, it will be dropped. So far this is not a concern in our database because all values appear in both tables. But if this is not the case, we want to specify what happens to the rows that do not find a match. That is where *outer joins* come in handy. There are two types. First, there is the *left join*, which keeps all the rows in the left table, and matches the corresponding rows from the right. If a row from the right table has a value without any match in the left table, it will be dropped. The *right join* is the mirror image of the left join. 

To further explore this types of joins, lets include a new table in the database with the economic outlook for different industries: 

```
CREATE TABLE industry_prospects (
  industry VARCHAR(20) PRIMARY KEY,
  outlook VARCHAR(20)
);
INSERT INTO industry_prospects VALUES
('Retail', 'Good'),
('Hospitality', 'Poor'),
('Logistics', 'Terrible'),
('Tourism', 'Great'),
('Events', 'Good');
```

If you compare this table to our `client` table, you will find that not all industries in the `client` table are present in the `industry_prospects` table and viceversa. Lets do an inner and a left join and see if the result is what we expect (try to think what the result should be before you run the code below).

```
SELECT client.client_id, client.industry, industry_prospects.industry, industry_prospects.outlook
FROM client
JOIN industry_prospects ON client.industry = industry_prospects.industry;

SELECT client.client_id, client.industry, industry_prospects.industry, industry_prospects.outlook
FROM client
LEFT JOIN industry_prospects ON client.industry = industry_prospects.industry;
```

The right join is performed using the similar key words `RIGHT JOIN`. In this case, the order how you specify the tables does matter. If you revert the order in a `LEFT JOIN` you will obtain the equivalent of a `RIGHT JOIN` and viceversa. 


### 5. Integrating SQLite and Python

You are now familiar with the concept of Relational Databases and the programming language used to operate with them. I hope that by now the strengths of this approach are apparent, and you understand why they are used in virtually any web application to store data. It should also be clear, though, that SQL is a domain-specific language that can manage databases but is not capable of doing much more. In general SQL is combined with other programming languages and dedicated exclusively to the management of data. In our case we will now learn how to interact with `.db` files using SQLite from within a Python file. This can be accomplished using the built-in Python module `sqlite3`. 

**The basics of sqlite3**

Lets look at a basic working example and use it to explain how it works (for more details go to the <a href = 'https://docs.python.org/3/library/sqlite3.html#how-to-use-placeholders-to-bind-values-in-sql-queries'> documentation </a>). The Connection object, in this case called `connection`, represents the connection to the `.db` file. If the file specified in the `.connect()` function exists in the current directory, it will be opened. If the file does not exist it will be created when the function is called. In our case, the previous line of code connects the file to the database we have been working all along. 

In [6]:
# make sure that 'school_database.db' is in the same directory 
# as this notebook before running this code.

import sqlite3

connection = sqlite3.connect('school_database.db')
cursor = connection.cursor()

result = cursor.execute("SELECT first_name, last_name FROM teacher WHERE teacher_id = 1")
result.fetchall()

cursor.execute("UPDATE teacher SET first_name = 'Nestor' WHERE teacher_id = 1")
connection.commit()

connection.close()


In order to run SQLite queries on this database we must create a cursor which, in this case, we call `cursor`. We are now ready to run SQLite code on the database by calling the cursor's function `cursor.execute()`, and writing a string of SQLite code as the first argument. In the example, we are doing a simple query that should return the name of the teacher with id 1. We are storing this output in the variable `result`. We can now use `.fetchone()` to obtain one row of output, `.fetchmany(n)` to obtain `n` rows of output or `.fetchall()` to obtain all of the output. In our particular case this is not relevant as our query will return a single row. 

We use `cursor.execute()` to run any type of SQLite code (All we have learned works here as well!). This includes code that modifies the database, such as the `UPDATE` statement that follows in our example. Notice that we are not saving the output into any variable, because this query returns no output. The changes these queries make are not automatically saved on the database. If one closes the connection or opens the queries the database from another program, he will not see that we changed the name of the first teacher. In order to permanently write the changes into the database we use `connection.commit()`. Finally, when one is done working with the database, the connection should be closed. 

**Further sqlite3 elements**

Often we will want to use Python variables in our SQLite code. To do so we use *placeholders*. The first approach is to use question marks: 

```
result = cursor.execute("SELECT * FROM client WHERE client_id = ?", (101,))
```

Here we put question marks in the elements of the query we want to take on Python variable values, and introduce a tuple with those Python variables as the second argument of the `execute` method. The tuple should contain the variables in the order they should replace question marks in the query. 

Another approach is to use named placeholders: 

```
result = cursor.execute("SELECT * FROM client WHERE client_id = :id", {'id':1})
```

In this case we use names preceded by a `:` in the query, and define them using a dictionary as the second argument of the `execute` method. Both methods are correct and work equally well. A third method that should be avoided is using string formatting, such as f-string, as they are very insecure when used in actual applications. 

Finally, we can use placeholders and the method `.executemany()` to run many similar queries in one go. If, for example, we want to insert four different new records into the `teacher` table:

```
data = [(7,'Tomas', 'Smith', 'MAN', None, '1990-11-12', 34650, '+447840921326'), (8, 'Maria', 'Johnson', 'ENG', None, '1985-09-25', 42000, '+447840921327'), (9, 'John', 'Doe', 'MAN', 'ENG', '1978-03-15', 50000, '+447840921328'), (10, 'Emily', 'Williams', 'ESP', None, '1995-07-08', 38000, '+447840921329')]

cursor.executemany("INSERT INTO teacher VALUES (?, ?, ?, ?, ?, ?, ?, ?)", data)
```

In this case, instead of passing one single tuple with the values to substitute in the placeholders of the query, we pass a list of tuples, one for each new record we want to create. It is important to note that `.executemany()` only works with queries that modify the database and will not work with queries that look up information (e.g. a `SELECT` statement)$^1$.

$^1$  In particular, it only works with DML statements. You can learn more about what these are and how one classifies SQL statements <a href = 'https://www.geeksforgeeks.org/sql-ddl-dql-dml-dcl-tcl-commands/#dcl-data-control-language'> here </a>.

**Questions**

-We want to have a Python dictionary that contains the tax_id and the telephone number for those teachers that give courses for an NGO. Use `sqlite3` to query `school_database.db` and obtain the desired information. You should be able to do this in one query. Then, use Python to create the dictionary containing this information. The keys of the dictionary should be the complete name of the teacher with an underscore separating the first and last name. The values of the dictionary should be a tuple containing the tax_id and the telephone number of the teacher. There should be no repeated entries in the dictionary. 

```
{'FirstName_LastName': (tax_id, telephone_number)...}
```

In [7]:
result = cursor.execute("""        
SELECT teacher.first_name, teacher.last_name, teacher.tax_id, teacher.phone_no
FROM teacher 
JOIN course ON course.teacher = teacher.teacher_id
WHERE course.client = (
SELECT client_id
FROM client
WHERE industry = 'NGO'
)
""")

result_list = result.fetchall()
tax_ngo_data = {}
for record in result_list:
    name = record[0] + '_' + record[1]    
    if name not in tax_ngo_data:
        tax_ngo_data[name] = (record[2], record[3])

tax_ngo_data

{'Niamh_Murphy': (67890, '+491231231232'),
 'Stefanie_Martin': (23456, '+491234567890')}

### 6. Exercise: Create your own relational database


In this exercise you are given the requirements for a database as well as some data in `.csv` format, and you will have to design and populate an SQL database. For this you will need to make use of what we have learned about RDs, SQLite and sqlite3 in Python. This is good practice for your *DE individual assignment*, where you will also need to design and population your own RD. 
 
> **Database requirements**: Musify is a music streaming service. Their servers contain many tracks users can listen to. Each of the tracks belongs to an album, which in turn belongs to one artist (there are no artists collaborations). Artists may, nevertheless, have multiple albums. Musify has created playlists to put together similar songs (e.g. classic songs, TV show songs). This playlists contain multiple tracks and tracks may be present in multiple playlists. 

> **Data**: You can find the provided data in the `exercise_data` folder that comes with this notebook. There are four `.csv` files.  

From this information and these data, you should create a database called `musify_database.db`. This should be a RD which correctly implements the database requirements, and where tables are populated with the data provided in the `.csv` files. 

**Hints and important steps**

- The best approach to solving this problem involves four steps. First, draw the ERM corresponding to this RD. What entities do we have, what are their relationship, what attributes do they have, which are foreign keys...? Secondly, read the data into Python using Pandas. You should clean it and modify so you have a DataFrame for each table you want to create. Third, create the database and its tables, following the design you have already layed out in your ERD. This step can be done using SQLite directory or within Python using sqlite3. Finally, you should populate the tables you have created. This step must be done in Python using sqlite3, and you can go about it in different ways. The tools needed are provided in section 5, but you should make use of your previous knowledge of Python (e.g. for loops, functions).
- When data is transferred from Python to a database, `sqlite3` attempts to do appropriate conversions (e.g. `int` $\rightarrow $ `INT`, `None` $\rightarrow$ `NULL`). Nevertheless, there are many different data types in Python (e.g. `numpy.int64`) that `sqlite3` cannot handle properly. A solution to this is to pass all the data as `strings`, because SQLite handles numbers and text in the same way. You should be careful though that `NaN` values are translated as `NULL`.
- As you have, hopefully, noticed from the requirements text, playlists and tracks have an N-to-M relationship. As we have seen, in SQL this sort of relationships are handled by means of a third table where each row is a 'link' between records. Note that we are missing this table at the moment, and we only have a column in `playlists` containing the tracks that are included in the playlist. From this, you should be able to arrange the data in such a way that the *connection* table can be populated.






**Answers**

The correct ERM for the exercise is: 

![ERD for the described requirements](diagrams/diagram_schools.jpg)



In [ ]:
# Answer
import pandas as pd
import sqlite3

# read dataframes
tracks_df = pd.read_csv(r'exercise_data/tracks.csv')
albums_df = pd.read_csv(r'exercise_data/albums.csv')
playlists_df = pd.read_csv(r'exercise_data/playlists.csv')
artists_df = pd.read_csv(r'exercise_data/artists.csv')

tracks_df = tracks_df.astype(str).replace('nan', None)
playlists_df = playlists_df.astype(str).replace('nan', None)
albums_df = albums_df.astype(str).replace('nan', None)
artists_df = artists_df.astype(str).replace('nan', None)

# arrange track_in_playlist table
def string_to_list(tracks):
    if isinstance(tracks, str):
        tracks = [element for element in tracks.split(',')]
    return tracks
    
playlists_df['Tracks'] = playlists_df['Tracks'].apply(string_to_list)

track_in_playlist = playlists_df.explode('Tracks')[['PlaylistId','Tracks']]
track_in_playlist.dropna(inplace = True)
track_in_playlist.rename(columns={'Tracks':'TrackId'}, inplace = True)

playlists_df = playlists_df[['PlaylistId', 'Name']]

# create tables 
connection = sqlite3.connect('musify_database.db')
cursor = connection.cursor()

cursor.execute("""
    CREATE TABLE playlist(
               PlaylistID INT PRIMARY KEY,
               Name TEXT 
    )
""")

cursor.execute("""
    CREATE TABLE artist(
               ArtistId INT PRIMARY KEY,
               Name TEXT
    )
""")

cursor.execute("""
    CREATE TABLE album(
               AlbumId INT PRIMARY KEY,
               Title TEXT,
               ArtistId INT,
               FOREIGN KEY(ArtistId) REFERENCES artist(ArtistId))
""")

cursor.execute("""
    CREATE TABLE track(
               TrackId INT PRIMARY KEY,
               Name TEXT,
               AlbumId INT,
               Composer TEXT,
               Milliseconds INT,
               Bytes INT,
               FOREIGN KEY(AlbumId) REFERENCES album(AlbumId))
""")

cursor.execute("""
    CREATE TABLE track_in_playlist(
               PlaylistId INT ,
               TrackId INT,
               PRIMARY KEY(PlaylistId, TrackId),
               FOREIGN KEY(PlaylistId) REFERENCES playlist(PlaylistId),
               FOREIGN KEY(TrackId) REFERENCES track(TrackId))
""")

# populate tables
def populate_table(table_name, data_frame):
    data = [tuple(data_frame.iloc[row,:]) for row in range(data_frame.shape[0])]

    number_of_variables = '?'
    for i in range(data_frame.shape[1]-1):
        number_of_variables += ',?'

    query = f'INSERT INTO {table_name} VALUES({number_of_variables})'

    cursor.executemany(query, data)

populate_table('track', tracks_df)
populate_table('artist', artists_df)
populate_table('album', albums_df)
populate_table('playlist', playlists_df)
populate_table('track_in_playlist',track_in_playlist)
connection.commit()

